In [ ]:
from vedo import *
import random
from matplotlib import colormaps
from vedo.colors import colors
from operator import itemgetter
import os

sorted_colors1 = sorted(colors.items(), key=itemgetter(1))
_cmaps = []
for sc in sorted_colors1:
    # Get the color name
    cname = sc[0]
    # Skip the color if it doesn't end in a number
    if cname[-1] not in "123456789":
        continue
    _cmaps.append(cname)
#_cmaps = list(colors.keys())
#_cmaps = ['blue5']
settings.tiff_orientation_type = 4 

random.shuffle(_cmaps)


_vol = Volume(dataurl + 'vase.vti')
_vol_norm = _vol.isosurface(4, flying_edges=False).normalize().pos(0,0,0).color('yellow5', 0.5)

num_of_slices = 20

for vol_index in range(100):
    _dir = f'temp/vase/0/fractured_{vol_index}'
    os.makedirs(_dir)

    v = vector(random.random(),random.random(), random.random())
    p = vector(0, 0, 0)  # axis passes through this point


    _vol_norm_rotated = _vol_norm.clone().rotate(45, axis=v, point=p).normalize().pos(0,0,0).color('blue5', 0.5)

    #l = Line(-v+p, v+p).lw(3).c('red')




    [xmin,xmax, ymin,ymax, zmin,zmax] = _vol_norm_rotated.bounds()
    slice_tickness = (ymax-ymin)/num_of_slices
    slice_size = (xmax-xmin, slice_tickness, zmax-zmin)

    #_cmaps = ['Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds','YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu','GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn']
    slice_list = []

    slice_color_alpha = 0.8
    for slice_index in range(num_of_slices):
        
        #slice_index = 3
        _color = _cmaps[slice_index]
        #_color = 'red5'
        bottom_box = Box(pos=(xmin,ymin,zmin), size=((xmax-xmin)*2, (slice_index*slice_tickness)*2, (zmax-zmin)*2))
        #bottom_box.color('green5', 0.5)
        top_box = Box(pos=(xmin,ymax ,zmin), size=((xmax-xmin)*2, ( (num_of_slices-slice_index-1)*slice_tickness)*2, (zmax-zmin)*2))
        #top_box.color('red5', 0.5)
        
        if slice_index == 0:
            _vol_norm_rotated_slice = _vol_norm_rotated.clone().cut_with_mesh(top_box, invert=True).color(_color, slice_color_alpha)
        elif  slice_index == num_of_slices - 1:
            _vol_norm_rotated_slice = _vol_norm_rotated.clone().cut_with_mesh(bottom_box, invert=True).color(_color, slice_color_alpha)
        else:
            _vol_norm_rotated_slice = _vol_norm_rotated.clone().cut_with_mesh(top_box, invert=True).cut_with_mesh(bottom_box, invert=True).color(_color, slice_color_alpha)
        _vol_norm_rotated_slice.write(f'{_dir}/piece_{slice_index}.obj')
        slice_list.append(_vol_norm_rotated_slice)
        #if True:
        #    break
        

#show(slice_list, __doc__, axes=1)